<a href="https://colab.research.google.com/github/Prateekg2050/Hindi_multi_Class_classification/blob/main/Final_train_test_score.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# install the libraries necessary for data wrangling, prediction and result analysis
import json
import numpy as np
import pandas as pd
import logging
import matplotlib.pyplot as plt
from sklearn import metrics
from sklearn.metrics import classification_report, confusion_matrix, f1_score,precision_score, recall_score
import torch
from numba import cuda
from sklearn.model_selection import train_test_split
from sklearn.dummy import DummyClassifier

In [ ]:
from numba import cuda
from itertools import islice
import time
from tqdm import tqdm

In [ ]:
!pip install -q transformers
!pip install -q simpletransformers
from simpletransformers.classification import ClassificationModel

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 91.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 kB 23.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 103.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.5/250.5 kB 15.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 5.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 108.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.7/468.7 kB 47.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 82.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 93.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 198.8/198.8 kB 24.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 kB 24.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import wandb
wandb.login()

<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [ ]:
# Clean the GPU cache

cuda.select_device(0)
cuda.close()
cuda.select_device(0)
torch.cuda.empty_cache()

In [ ]:
train_df = pd.read_csv('./our_dataset_merged.csv')
print(train_df)

test_df = train_df[900:1000]
test_df = test_df.drop(['label'],axis = 1)
print(test_df)

                                                   text  \
0     #29: बोत्सवाना + 12.1% अर्थव्यवस्था: बोत्सवाना...   
1     'पदक के अलावा कुछ भी निराशा होने वाला है' अनुभ...   
2     'बी' आर्क ... "मेरा मतलब है, मैं ध्यान देने मे...   
3     'मिडसोमर की मस्ती, लेकिन इस कहानी को बताने की ...   
4     'मिडसोमर की मस्ती, लेकिन इस कहानी को बताने की ...   
...                                                 ...   
1091  इतनी दूर न जाना कि लौट न सको,\n\nइतना व्यस्त न...   
1092              रह गयी है कुछ कमी तो, शिकायत क्या है,   
1093          इस जहाँ में सब अधूरा है, मुकम्मल क्या है।   
1094      रास्ते कहाँ ख़त्म होते हैं जिंदगी के सफर में,   
1095           मंज़िल तो वही है जहाँ ख्वाहिशें थम जाएँ।   

                        label  
0                        News  
1                        News  
2                       Other  
3     Information/Explanation  
4       Opinion/Argumentation  
...                       ...  
1091            Prose/Lyrical  
1092            Prose/Lyrical  
109

In [ ]:
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"

In [ ]:
# Create a list of labels
LABELS = train_df.label.unique().tolist()
LABELS
train_df['label']=train_df['label'].replace(['Other', 'Information/Explanation', 'News', 'Instruction', 'Opinion/Argumentation', 'Forum', 'Prose/Lyrical', 'Legal', 'Promotion'],[0,1,2,3,4,5,6,7,8])
dev_df = train_df[:100]
print(dev_df)

                                                 text  label
0   #29: बोत्सवाना + 12.1% अर्थव्यवस्था: बोत्सवाना...      2
1   'पदक के अलावा कुछ भी निराशा होने वाला है' अनुभ...      2
2   'बी' आर्क ... "मेरा मतलब है, मैं ध्यान देने मे...      0
3   'मिडसोमर की मस्ती, लेकिन इस कहानी को बताने की ...      1
4   'मिडसोमर की मस्ती, लेकिन इस कहानी को बताने की ...      4
..                                                ...    ...
95  अभिव्यक्ति की स्वतंत्रता एक अवधारणा है कि व्यक...      4
96  अमेज़ॅन की फिल्म और टीवी सब्सक्रिप्शन सेवा जिस...      1
97  अमेरिकी शेयर उन्नत हुए, स्टैंडर्ड एंड पूअर्स 5...      2
98  अराउंड द वर्ल्ड इन अस्सी डेज द्वारा: जूल्स वर्...      1
99  अरे दुनिया।हाँ मैं उम्मीद कर रहा था कि इस समय ...      5

[100 rows x 2 columns]


In [ ]:
# Initialize Wandb
wandb.init(project="Hindi-hyperparameter-search", name="training-and-saving-the-model")

wandb: Currently logged in as: prateekg2050 (lazy-ai). Use `wandb login --relogin` to force relogin


In [ ]:
# Calculate how many steps will each epoch have
# Num steps in epoch = training samples / batch size
steps_per_epoch = int(1000/8)
steps_per_epoch

125

In [ ]:
# Create a TransformerModel and evaluate during training
epoch = 30

roberta_base_model = ClassificationModel(
        "xlmroberta", "TajaKuzman/xlm-roberta-base-multilingual-text-genres",
        use_cuda=True,
        args= {
            "overwrite_output_dir": True,
            "num_train_epochs": epoch,
            "train_batch_size":8,
            "learning_rate": 1e-5,
            # Use these parameters if you want to evaluate during training
            "evaluate_during_training": True,
            "evaluate_during_training_steps": steps_per_epoch*10,
            "evaluate_during_training_verbose": True,
            "use_cached_eval_features": True,
            'reprocess_input_data': True,
            # "labels_list": LABELS,
            # The following parameters (no_cache, no_save) are commented out if I want to save the model
            "no_cache": True,
            # Disable no_save: True if you want to save the model
            "no_save": True,
            "max_seq_length": 512,
            "save_steps": -1,
            # Only the trained model will be saved - to prevent filling all of the space
            "save_model_every_epoch":False,
            "wandb_project": 'Hindi-hyperparameter-search',
            "silent": True,
            }
        )

# Train the model and evaluate during training
roberta_base_model.train_model(train_df, eval_df = dev_df)


/usr/local/lib/python3.9/dist-packages/simpletransformers/classification/classification_model.py:486: UserWarning: use_multiprocessing automatically disabled as xlmroberta fails when using multiprocessing for feature conversion.
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/simpletransformers/classification/classification_model.py:612: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  warnings.warn(


<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


/usr/local/lib/python3.9/dist-packages/torch/optim/lr_scheduler.py:139: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn("Detected call of `lr_scheduler.step()` before `optimizer.step()`. "
/usr/local/lib/python3.9/dist-packages/simpletransformers/classification/classification_model.py:1454: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/simpletransformers/classification/classification_model.py:1454: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  warnings.w

In [ ]:
def predict(dataframe, dataframe_column, final_file):
  """
    The function takes the dataframe with text in column dataframe_column, creates batches of 8,
    and applies genre predictions on batches, for faster prediction.
    It saves the file with text and predictions with the final_file name.

    Args:
      - dataframe (pandas Dataframe): specify the dataframe
      - dataframe_column (str): specify which column in the dataframe has texts to which you want to predict genres, e.g. ("docs")
      - final_file: the name of the final file with predictions

  """
    # Split the dataframe into batches
    # Create batches of text
  def chunk(arr_range, arr_size):
      arr_range = iter(arr_range)
      return iter(lambda: tuple(islice(arr_range, arr_size)), ())

  batches_list = list(chunk(dataframe[dataframe_column], 8))

  batches_list_new = []

  for i in batches_list:
      batches_list_new.append(list(i))

  print("The dataset is split into {} batches of {} texts.".format(len(batches_list_new),len(batches_list_new[0])))

  y_pred = []

  print("Prediction started.")
  start_time = time.time()

  for i in tqdm(batches_list_new):
      output = roberta_base_model.predict(i)
      current_y_pred = [roberta_base_model.config.id2label[i] for i in output[0]]

      for i in current_y_pred:
          y_pred.append(i)

  prediction_time = round((time.time() - start_time)/60,2)

  print("\n\nPrediction completed. It took {} minutes for {} instances - {} minutes per one instance.".format(prediction_time, dataframe.shape[0], prediction_time/dataframe.shape[0]))

  dataframe["X-GENRE"] = y_pred

  # Save the new dataframe which contains the y_pred values as well
  dataframe.to_csv("{}".format(final_file), sep="\t")

  return dataframe


In [ ]:
test_pred_df = predict (test_df, "text", "Hindi-test-sample-predicted.csv")

The dataset is split into 13 batches of 8 texts.
Prediction started.


100%|██████████| 13/13 [00:18<00:00,  1.39s/it]



Prediction completed. It took 0.3 minutes for 100 instances - 0.003 minutes per one instance.


In [ ]:
test_pred_df

,text,X-GENRE
900,संबंधित विषय 2 में से 1. इजराइल के प्रधान मंत्...,News
901,संयुक्त राष्ट्र में बलि का बकरा के रूप में इज़...,News
902,संरचनाओं के परिणाम: एक लंबा समय इतना बुरा नहीं...,News
903,संशोधित एसटीटीएम पुस्तक आ चुकी है! आपके पास दो...,Other
904,सच्चा नेतृत्व लोगों को ऐसी जगह ले जाने के बारे...,Opinion/Argumentation
...,...,...
995,हाइलाइट मोबाइल फोन की पहुंच दुनिया की तीन चौथा...,News
996,हाउस इन द कंट्री: अंटार्कटिका हर सुबह उस नज़ार...,Information/Explanation
997,हाय दोस्तों मैं दो अद्भुत साइटों पर दो उपहार द...,Opinion/Argumentation
998,हाय दोस्तों मैं दो अद्भुत साइटों पर दो उपहार द...,Opinion/Argumentation


In [ ]:
from sklearn.metrics import accuracy_score,confusion_matrix,classification_report


test_pred_df['X-GENRE1'] = test_pred_df['X-GENRE'].replace(['Other', 'Information/Explanation', 'News', 'Instruction', 'Opinion/Argumentation', 'Forum', 'Prose/Lyrical', 'Legal', 'Promotion'],[0,1,2,3,4,5,6,7,8])
# print(test_pred_df)
print("Accuracy Score\n", accuracy_score(train_df[900:1000]['label'],test_pred_df['X-GENRE1']))
print("\n\nConfusion matrix\n" , confusion_matrix(train_df[900:1000]['label'],test_pred_df['X-GENRE1']))
print("\n\nClassification Report\n" , classification_report(train_df[900:1000]['label'],test_pred_df['X-GENRE1']))

Accuracy Score
 0.81


Confusion matrix
 [[11  0  0  0 10  0]
 [ 0 16  0  0  1  0]
 [ 0  0 28  0  2  0]
 [ 0  0  0  2  0  0]
 [ 5  1  0  0 18  0]
 [ 0  0  0  0  0  6]]


Classification Report
               precision    recall  f1-score   support

           0       0.69      0.52      0.59        21
           1       0.94      0.94      0.94        17
           2       1.00      0.93      0.97        30
           3       1.00      1.00      1.00         2
           4       0.58      0.75      0.65        24
           5       1.00      1.00      1.00         6

    accuracy                           0.81       100
   macro avg       0.87      0.86      0.86       100
weighted avg       0.82      0.81      0.81       100



In [ ]:
#this cell is used to save pickel file of model

# import pickle       
# with open('model1','wb') as f:
#   pickle.dump(roberta_base_model,f)

In [ ]:
#  this cell is used to connect to google drive to transfer your saved model
# from google.colab import drive       
# drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# this cell used to load pickel file
# with open('model1','rb') as f:
#   model2 = pickle.load(f)

cp: cannot stat '/content/drive/MyDrive/torch_model1.pt': No such file or directory


In [ ]:
def predict(dataframe, dataframe_column, final_file):
  """
    The function takes the dataframe with text in column dataframe_column, creates batches of 8,
    and applies genre predictions on batches, for faster prediction.
    It saves the file with text and predictions with the final_file name.

    Args:
      - dataframe (pandas Dataframe): specify the dataframe
      - dataframe_column (str): specify which column in the dataframe has texts to which you want to predict genres, e.g. ("docs")
      - final_file: the name of the final file with predictions

  """
    # Split the dataframe into batches
    # Create batches of text
  def chunk(arr_range, arr_size):
      arr_range = iter(arr_range)
      return iter(lambda: tuple(islice(arr_range, arr_size)), ())

  batches_list = list(chunk(dataframe[dataframe_column], 8))

  batches_list_new = []

  for i in batches_list:
      batches_list_new.append(list(i))

  print("The dataset is split into {} batches of {} texts.".format(len(batches_list_new),len(batches_list_new[0])))

  y_pred = []

  print("Prediction started.")
  start_time = time.time()

  for i in tqdm(batches_list_new):
      output = model2.predict(i)
      current_y_pred = [model2.config.id2label[i] for i in output[0]]

      for i in current_y_pred:
          y_pred.append(i)

  prediction_time = round((time.time() - start_time)/60,2)

  print("\n\nPrediction completed. It took {} minutes for {} instances - {} minutes per one instance.".format(prediction_time, dataframe.shape[0], prediction_time/dataframe.shape[0]))

  dataframe["X-GENRE"] = y_pred

  # Save the new dataframe which contains the y_pred values as well
  dataframe.to_csv("{}".format(final_file), sep="\t")

  return dataframe

In [ ]:
test_pred_df = predict (test_df, "text", "Hindi-test-sample-predicted.csv")

The dataset is split into 13 batches of 8 texts.
Prediction started.


100%|██████████| 13/13 [00:20<00:00,  1.60s/it]



Prediction completed. It took 0.35 minutes for 100 instances - 0.0034999999999999996 minutes per one instance.


In [ ]:
test_pred_df

,text,X-GENRE,X-GENRE1
900,संबंधित विषय 2 में से 1. इजराइल के प्रधान मंत्...,News,2
901,संयुक्त राष्ट्र में बलि का बकरा के रूप में इज़...,News,2
902,संरचनाओं के परिणाम: एक लंबा समय इतना बुरा नहीं...,News,2
903,संशोधित एसटीटीएम पुस्तक आ चुकी है! आपके पास दो...,Other,0
904,सच्चा नेतृत्व लोगों को ऐसी जगह ले जाने के बारे...,Opinion/Argumentation,4
...,...,...,...
995,हाइलाइट मोबाइल फोन की पहुंच दुनिया की तीन चौथा...,News,2
996,हाउस इन द कंट्री: अंटार्कटिका हर सुबह उस नज़ार...,Information/Explanation,1
997,हाय दोस्तों मैं दो अद्भुत साइटों पर दो उपहार द...,Opinion/Argumentation,4
998,हाय दोस्तों मैं दो अद्भुत साइटों पर दो उपहार द...,Opinion/Argumentation,4


In [ ]:
from sklearn.metrics import accuracy_score,confusion_matrix,classification_report


test_pred_df['X-GENRE1'] = test_pred_df['X-GENRE'].replace(['Other', 'Information/Explanation', 'News', 'Instruction', 'Opinion/Argumentation', 'Forum', 'Prose/Lyrical', 'Legal', 'Promotion'],[0,1,2,3,4,5,6,7,8])
# print(test_pred_df)
print("Accuracy Score\n", accuracy_score(train_df[900:1000]['label'],test_pred_df['X-GENRE1']))
print("\n\nConfusion matrix\n" , confusion_matrix(train_df[900:1000]['label'],test_pred_df['X-GENRE1']))
print("\n\nClassification Report\n" , classification_report(train_df[900:1000]['label'],test_pred_df['X-GENRE1']))

Accuracy Score
 0.81


Confusion matrix
 [[11  0  0  0 10  0]
 [ 0 16  0  0  1  0]
 [ 0  0 28  0  2  0]
 [ 0  0  0  2  0  0]
 [ 5  1  0  0 18  0]
 [ 0  0  0  0  0  6]]


Classification Report
               precision    recall  f1-score   support

           0       0.69      0.52      0.59        21
           1       0.94      0.94      0.94        17
           2       1.00      0.93      0.97        30
           3       1.00      1.00      1.00         2
           4       0.58      0.75      0.65        24
           5       1.00      1.00      1.00         6

    accuracy                           0.81       100
   macro avg       0.87      0.86      0.86       100
weighted avg       0.82      0.81      0.81       100



In [ ]:
#this cell is used to send torch file to google drive
!cp torch_model1.pt /content/drive/MyDrive

In [ ]:
# this cell is used to save torch file of model
import torch
torch.save(roberta_base_model, 'torch_model1.pt')